In [3]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Get Notes from Music Files

In [4]:
notes = []


for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file) #convert the file into stream.Score Object

    print("Parsing %s" % file)

    notes_to_parse = None


    
    
    
#     try: # file has instrument parts
#         s2 = instrument.partitionByInstrument(midi)
#         notes_to_parse = s2.parts[0].recurse() 
#     except: # file has notes in a flat structure
#         notes_to_parse = midi.flat.notes
    
    
    
    
    
    # unroll / flat the elements (notes/chords) into the list, because elements are sometimes in list of list form.
    notes_to_parse = midi.flat.notes   
    
    
    
    
    # At this point, we have notes_to_parse, which is a Iterator for all the Notes/Chords
    
    
    
    
    for element in notes_to_parse:
        
        # If the element is a Note, then store it's pitch
        if isinstance(element, note.Note): 
            notes.append(str(element.pitch))
            
        # If the element is a Chord, split each note of the chord and join them with +
        elif isinstance(element, chord.Chord): 
            notes.append('+'.join(str(n) for n in element.normalOrder))

Parsing midi_songs/0fithos.mid
Parsing midi_songs/FF3_Third_Phase_Final_(Piano).mid
Parsing midi_songs/Kingdom_Hearts_Traverse_Town.mid
Parsing midi_songs/ff7-mainmidi.mid
Parsing midi_songs/relmstheme-piano.mid
Parsing midi_songs/rufus.mid
Parsing midi_songs/pkelite4.mid
Parsing midi_songs/Rydia_pc.mid
Parsing midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Parsing midi_songs/thoughts.mid
Parsing midi_songs/JENOVA.mid
Parsing midi_songs/lurk_in_dark.mid
Parsing midi_songs/Life_Stream.mid
Parsing midi_songs/ff4pclov.mid
Parsing midi_songs/Ff7-One_Winged.mid
Parsing midi_songs/decisive.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/z_aeristhemepiano.mid
Parsing midi_songs/FF8_Shuffle_or_boogie_pc.mid
Parsing midi_songs/FF6epitaph_piano.mid
Parsing midi_songs/dayafter.mid
Parsing midi_songs/HighwindTakestotheSkies.mid
Parsing midi_songs/ff11_awakening_piano.mid
Parsing midi_songs/In_Zanarkand.mid
Parsing midi_songs/Gold_Silver_Rival_Battle.mid
Parsing m

**Demo of Above Code with single File**

In [4]:
midi = converter.parse("midi_songs/mining.mid") #convert the file into stream.Score Object

In [5]:
midi

<music21.stream.Score 0x16eea05bdd8>

In [6]:
notes_to_parse = midi.flat.notes

In [7]:
len(notes_to_parse)

237

In [8]:
#  Here offset is 0.5 everytime
for i in notes_to_parse:
    print(i, i.offset)

<music21.chord.Chord D3 A3> 0.0
<music21.chord.Chord E3 B3> 0.5
<music21.note.Note B> 1.0
<music21.note.Note B> 1.0
<music21.chord.Chord D3 A3> 1.5
<music21.chord.Chord E3 B3> 2.0
<music21.chord.Chord D3 A3> 3.0
<music21.chord.Chord E3 B3> 3.5
<music21.note.Note B> 4.0
<music21.chord.Chord D3 A3> 4.5
<music21.chord.Chord E3 B3> 5.0
<music21.chord.Chord D3 A3> 6.0
<music21.chord.Chord E3 B3> 6.5
<music21.note.Note B> 7.0
<music21.note.Note B> 7.0
<music21.chord.Chord D3 A3> 7.5
<music21.chord.Chord E3 B3> 8.0
<music21.chord.Chord D3 A3> 9.0
<music21.chord.Chord E3 B3> 9.5
<music21.note.Note B> 10.0
<music21.chord.Chord D3 A3> 10.5
<music21.chord.Chord E3 B3> 11.0
<music21.note.Note G> 11.5
<music21.note.Note A> 11.75
<music21.note.Note B> 12.0
<music21.chord.Chord D3 A3> 12.0
<music21.chord.Chord E3 B3> 12.5
<music21.note.Note B> 13.0
<music21.chord.Chord D3 A3> 13.5
<music21.chord.Chord E3 B3> 14.0
<music21.note.Note A> 14.5
<music21.note.Note G> 14.75
<music21.note.Note F#> 15.0
<musi

In [9]:
notes_demo= []
for element in notes_to_parse:
    # If the element is a Note, then store it's pitch
    if isinstance(element, note.Note): 
        notes_demo.append(str(element.pitch))

    # If the element is a Chord, split each note of the chord and join them with dot
    elif isinstance(element, chord.Chord): 
        notes_demo.append('+'.join(str(n) for n in element.normalOrder))

In [10]:
# got the notes and chords
print(notes_demo[:15])

['9+2', '11+4', 'B5', 'B2', '9+2', '11+4', '9+2', '11+4', 'B2', '9+2', '11+4', '9+2', '11+4', 'B5', 'B2']


In [0]:
with open('data/notes_easy', 'wb') as filepath:
    pickle.dump(notes, filepath)

In [41]:
with open('data/notes_easy', 'rb') as filepath:
    notes = pickle.load(filepath)

In [12]:
n_vocab = len(set(notes))

In [13]:
print("Total notes- ",len(notes))
print("Unique Notes- ",n_vocab)

Total notes-  60498
Unique Notes-  359


In [14]:
# notes is a list of all the notes in each music file - 60000
print(notes[:100])

['4+9', 'E2', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', '11+4', '4+9', '11+4', '4+9', '4+9', '4+9', '4+9', '4+9', '0+4', 'E2', '4+9', '0+4', '4+9', '4+9', '4+9', '4+9', '4+9', '9+2', '4+9', '9+2', '9+2', '4+9', '4+9', '4+9', '4+9', '4+9', '4+9', 'E2', '4+9', '4+9', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '5+11', '4+9', '5+11', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '9+0', 'E2', '4+9', '9+0', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '11+2', '4+9', '11+2', '11+2', '4+9', '4+9', '4+9', 'E5', 'F5', 'G#5', 'A5', '4+9', '4+9', '3+7+11', 'E-2', '3+7+11', 'B2', 'G2', '1+5+9', 'F#2', '1+5+9', '3+7+11', 'E-2', '3+7+11', 'G2', 'B2', 'E-3']


# Prepare Sequential Data for LSTM

In [15]:
#  How many elements LSTM input should consider
sequence_length = 100

In [16]:
# ALl the unique elements in a sorted manner
pitchnames = sorted(set(item for item in notes))

In [17]:
# Mapping between note to int value
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
print(note_to_int)

{'0': 0, '0+1': 1, '0+1+5': 2, '0+1+6': 3, '0+2': 4, '0+2+3+7': 5, '0+2+4+7': 6, '0+2+5': 7, '0+2+6': 8, '0+2+7': 9, '0+3': 10, '0+3+5': 11, '0+3+5+8': 12, '0+3+6': 13, '0+3+6+8': 14, '0+3+6+9': 15, '0+3+7': 16, '0+4': 17, '0+4+5': 18, '0+4+6': 19, '0+4+7': 20, '0+5': 21, '0+5+6': 22, '0+6': 23, '1': 24, '1+2': 25, '1+2+4+6+8+10': 26, '1+2+6': 27, '1+2+6+8': 28, '1+3': 29, '1+3+5': 30, '1+3+5+8': 31, '1+3+6': 32, '1+3+7': 33, '1+3+8': 34, '1+4': 35, '1+4+6': 36, '1+4+6+9': 37, '1+4+7': 38, '1+4+7+10': 39, '1+4+7+9': 40, '1+4+8': 41, '1+5': 42, '1+5+8': 43, '1+5+9': 44, '1+6': 45, '1+7': 46, '10': 47, '10+0': 48, '10+0+2+5': 49, '10+0+3': 50, '10+0+4': 51, '10+0+5': 52, '10+1': 53, '10+1+3': 54, '10+1+3+5+6': 55, '10+1+3+6': 56, '10+1+4': 57, '10+1+4+6': 58, '10+1+5': 59, '10+11': 60, '10+11+3': 61, '10+11+3+5': 62, '10+2': 63, '10+2+3': 64, '10+2+4': 65, '10+2+5': 66, '10+3': 67, '11': 68, '11+0': 69, '11+0+4': 70, '11+0+4+6': 71, '11+0+4+7': 72, '11+0+5': 73, '11+1': 74, '11+1+4': 75,

In [18]:
network_input = []
network_output = []

In [19]:
# Make the data for LSTM Network
# Each Node will contain 100 input units
# Output will be the next unit in notes list

for i in range(0, len(notes) - sequence_length, 1):
    
    sequence_in = notes[i:i + sequence_length] # contains 100 values 
    sequence_out = notes[i + sequence_length] # containes next values for these 100's
    
    # Since NN works with numeric data only, append the int values for inputs and outputs.
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

In [20]:
# No. of examples for our network

n_patterns = len(network_input)
print(n_patterns)

60398


In [21]:
# ight now network_input is lits, but reshape it into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
print(network_input.shape)

(60398, 100, 1)


In [22]:
normalised_network_input = network_input/float(n_vocab)

In [23]:
# network_output are the classes, encode one_hot_vector
network_output = np_utils.to_categorical(network_output)

In [24]:
print(network_output.shape)
print(normalised_network_input.shape)

(60398, 359)
(60398, 100, 1)


# Create Model

In [25]:
""" create the structure of the neural network """
model = Sequential()
model.add(LSTM(512,
            input_shape=(network_input.shape[1], network_input.shape[2]),
            return_sequences=True ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
__________

In [22]:
""" train the neural network """

filepath = "model/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min')

callbacks_list = [checkpoint]

model_his = model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)

W0806 19:54:23.876171 140548401108864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
60398/60398 [==============================] - 396s 7ms/step - loss: 4.5791
Epoch 2/100
60398/60398 [==============================] - 395s 7ms/step - loss: 4.3746
Epoch 3/100
60398/60398 [==============================] - 408s 7ms/step - loss: 4.2257
Epoch 4/100
60398/60398 [==============================] - 390s 6ms/step - loss: 4.0589
Epoch 5/100
60398/60398 [==============================] - 388s 6ms/step - loss: 3.8828
Epoch 6/100
60398/60398 [==============================] - 391s 6ms/step - loss: 3.7038
Epoch 7/100
60398/60398 [==============================] - 393s 7ms/step - loss: 3.5276
Epoch 8/100
60398/60398 [==============================] - 391s 6ms/step - loss: 3.3695
Epoch 9/100
60398/60398 [==============================] - 390s 6ms/step - loss: 3.2217
Epoch 10/100
60398/60398 [==============================] - 389s 6ms/step - loss: 3.1012
Epoch 11/100
60398/60398 [==============================] - 390s 6ms/step - loss: 2.9938
Epoch 12/100
60398/60398 [====

KeyboardInterrupt: ignored

In [27]:
model = load_model("model/weights-improvement-60-1.1406-bigger.hdf5")

Instructions for updating:
Use tf.cast instead.


# Predictions

In [42]:
#  This is done beacuse we need network_input as list, and we had converted it into ndarray to feed into network
#  so, to get back the list.

sequence_length = 100
network_input = []
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])

In [62]:
""" Generate notes from the neural network based on a sequence of notes """

# pick a random sequence from the input as a starting point for the prediction
start = numpy.random.randint(0, len(network_input)-1)

# Mapping from int to note
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

# initial pattern
pattern = network_input[start]
prediction_output = []

# generate 200 notes
for note_index in range(200):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)

    prediction = model.predict(prediction_input, verbose=0)

    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    
    # Remove the first value, and append the recent value.. 
    # This way input is moving forward step-by-step with time..
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [63]:
#  Our newly generated song (mix of chords and notes)
print(prediction_output)

['F5', '2+5', '2+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', 'G5', 'G5', '2+5', '2+5', '2+5', '2+5', 'F5', '0+5', '2+5', '2+5', '2+5', '2+5', '0+5', '2+5', 'G5', '0+5', '0+4', '0+4', '2+5', '2+5', 'G5', '2+5', 'B4', '2+5', '2+5', 'G5', '0+5', '0+5', '0+5', 'B4', '2+5', '2+5', '2+5', '2+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', 'F5', '2+5', '2+5', '2+5', '2+5', '2+5', '2+5', '2+5', '0+5', '2+5', '0+5', '0+5', '2+5', '0+5', '0+5', '0+5', 'C5', '0+5', 'C5', '2+5', '2+5', '2+5', '2+5', '0+5', '2+5', '0+5', '2+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', '0+5', 'F5', '0+5', 'G5', '0+5', '2+5', '0+5', '2+5', '2+5', '2+5', '2+5', '2+5', '2+5', '0+5', '2+5', 'B4', '0+5', '0+5', 'F#4', 'B-2', 'E-5', 'E3', 'B5', 'B4', 'G5', '0+5', '2+5', '0+5', 'B-2', 'B-2', 'B-2', '0+5', '0+5', 'F5', 'G5', '0+5', '0+5', '0+5', '0+5', '0+4', '0+5', 'E3', '0+5', '0+5', 'F5', '0+5', '2+5', '0+5', '0+5', '0+5', '0+5', '0+5', '9+0', '10+3', '10+3', 'E3', 'C3', 'E3', 'D2', 'B5', 'G5', 'G5', 'F5', 'G5', 

# Create Midi File

In [64]:
""" convert the output from the prediction to notes and create a midi file from the notes """

offset = 0 # Time
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    
    # if pattern is a chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+') # split all the notes from a chord
        notes = []
        for current_note in notes_in_chord: 
            new_note = note.Note(int(current_note)) # create Note object for each note in the chord
            new_note.storedInstrument = instrument.Piano() 
            notes.append(new_note) # list of Notes()
            
        new_chord = chord.Chord(notes)  # Create the Chord() from the list of notes.
        new_chord.offset = offset # set offset to the element
        output_notes.append(new_chord) 
        
    # if pattern is a note
    else:
        new_note = note.Note(pattern) #  create Note object
        new_note.offset = offset # set offset  (basically timestamp)
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note) 

    # increase offset each iteration so that notes do not stack (time stamp)
    offset += 0.5

In [65]:
output_notes[:50]

[<music21.note.Note F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C F>,
 <music21.note.Note G>,
 <music21.note.Note G>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord D F>,
 <music21.note.Note G>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C E>,
 <music21.chord.Chord C E>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note G>,
 <music21.chord.Chord D F>,
 <music21.note.Note B>,
 <music21.chord.Chord D F>,
 <music21.chord.Chord D F>,
 <music21.note.Note G>,
 <music21.chord.Chord C F>,
 <music21.chord.Chord C 

In [69]:
#  Create a Steam Object from our generated Notes, and write to the file. 
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='output_songs/test_output.mid')

'output_songs/test_output.mid'

### Let's Play Our Music

In [70]:
midi_stream.show('midi')